<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Build Smarter AI Apps: Empower LLMs with LangChain**


### Installing required libraries


In [ ]:
%%capture
!pip install --force-reinstall --no-cache-dir tenacity==8.2.3 --user
!pip install "ibm-watsonx-ai==1.0.8" --user
!pip install "ibm-watson-machine-learning==1.0.367" --user
!pip install "langchain-ibm==0.1.7" --user
!pip install "langchain-community==0.2.10" --user
!pip install "langchain-experimental==0.0.62" --user
!pip install "langchainhub==0.1.18" --user
!pip install "langchain==0.2.11" --user
!pip install "pypdf==4.2.0" --user
!pip install "chromadb==0.4.24" --user

After you install the libraries, restart your kernel by clicking the **Restart the kernel** icon as shown in the following screenshot:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/kql9mdh7bKPx6uWW0-AP-Q/restart-kernel.jpg" style="margin:1cm;width:90%;border:1px solid grey" alt="Restart kernel">


### Importing required libraries

The following code imports the required libraries:


In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

The following code will construct a `mixtral-8x7b-instruct-v01` watsonx.ai inference model object:


In [ ]:
model_id = 'mistralai/mixtral-8x7b-instruct-v01'

parameters = {
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.2, # this randomness or creativity of the model's responses
}

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com"
}

project_id = "skills-network"

model = ModelInference(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

Let's use a simple example to let the model generate some text:


In [ ]:
msg = model.generate("In today's sales meeting, we ")
print(msg['results'][0]['generated_text'])

### Chat model


Chat models support assigning distinct roles to conversation messages, helping to distinguish messages from AI, users, and instructions such as system messages.


To enable the LLM from watsonx.ai to work with LangChain, you need to wrap the LLM using `WatsonLLM()`. This wrapper converts the LLM into a chat model, which allows the LLM to integrate seamlessly with LangChain's framework for creating interactive and dynamic AI applications.


In [ ]:
mixtral_llm = WatsonxLLM(model = model)

### Exercise 1 
#### **Compare Model Responses with Different Parameters**

Watsonx.ai provides access to several foundational models. In the previous section you used `mistralai/mixtral-8x7b-instruct-v01`. Try using another foundational model, such as `'meta-llama/llama-3-3-70b-instruct'`.


**Instructions**:

1. Create two instances, one instance for the Mixtral model and one instance for the Llama model. You can also adjust each model's creativity with different temperature settings.
2. Send identical prompts to each model and compare the responses.
3. Try at least 3 different types of prompts.

Check out these prompt types:

| Prompt type |   Prompt Example  |
|------------------- |--------------------------|
| **Creative writing**  | "Write a short poem about artificial intelligence." |
| **Factual questions** |  "What are the key components of a neural network?"  |
| **Instruction-following**  | "List 5 tips for effective time management." |

Then document your observations on how temperature affects:

- Creativity compared to consistency
- Variation between multiple runs
- Appropriateness for different tasks





In [ ]:
# Define different parameter sets
parameters_creative = {
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.8,  # Higher temperature for more creative responses
}

parameters_precise = {
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.1,  # Lower temperature for more deterministic responses
}

# Define the model ID for mixtral-8x7b-instruct-v01
mixtral='mistralai/mixtral-8x7b-instruct-v01'

# Define the model ID for llama-3-3-70b-instruct
llama='meta-llama/llama-3-3-70b-instruct'

mixtral_creative = ModelInference(
    model_id=mixtral,
    params=parameters_creative,
    credentials=credentials,
    project_id=project_id
)

mixtral_precise = ModelInference(
    model_id=mixtral,
    params=parameters_precise,
    credentials=credentials,
    project_id=project_id
)

llama_creative = ModelInference(
    model_id=llama,
    params=parameters_creative,
    credentials=credentials,
    project_id=project_id
)

llama_precise = ModelInference(
    model_id=llama,
    params=parameters_precise,
    credentials=credentials,
    project_id=project_id
)

mixtral_llm_creative = WatsonxLLM(model=mixtral_creative)
mixtral_llm_precise = WatsonxLLM(model=mixtral_precise)
llama_llm_creative = WatsonxLLM(model=llama_creative)
llama_llm_precise = WatsonxLLM(model=llama_precise)

prompts = [
    "What is the capital of France?",
    "Which month follows June?",
    "Generate a short story about a cat."
]

for prompt in prompts:
    print(f"=== {prompt} ===")
    print("Mixtral Creative:")
    print(mixtral_llm_creative.invoke(prompt))
    print("Mixtral Precise:")
    print(mixtral_llm_precise.invoke(prompt))
    print("Llama Creative:")
    print(llama_llm_creative.invoke(prompt))
    print("Llama Precise:")
    print(llama_llm_precise.invoke(prompt))
    print()

### Exercise 2 
#### **Creating and Using a JSON Output Parser**

Now let's implement a simple JSON output parser to structure the responses from your LLM.

**Instructions:**  

You'll complete the following steps:

1. Import the necessary components to create a JSON output parser.
2. Create a prompt template that requests information in JSON format (hint: use the provided template).
3. Build a chain that connects your prompt, LLM, and JSON parser.
4. Test your parser using at least three different inputs.
5. Access and display specific fields from the parsed JSON output.
6. Verify that your output is properly structured and accessible as a Python dictionary.


In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Create your JSON parser
json_parser = JsonOutputParser()

# Create the format instructions
format_instructions = """RESPONSE FORMAT: Return ONLY a JSON object with no text before or after. The JSON must have these keys:
{
  "title": "movie title",
  "director": "director name",
  "year": year as number,
  "genre": "movie genre"
}

IMPORTANT: Your entire response must be valid JSON. Do not include any explanatory text outside the JSON structure."""

# Create prompt template with instructions
prompt_template = PromptTemplate(
    template="""You are a JSON-generating assistant that only outputs valid JSON.

Task: Generate information about the movie "{movie_name}" in JSON format.

{format_instructions}""",
    input_variables=["movie_name"],
    partial_variables={"format_instructions": format_instructions}
)

# Create the chain
movie_chain = prompt_template | mixtral_llm | json_parser

# Test with a movie name
movie_name = "The Matrix"
result = movie_chain.invoke({"movie_name": movie_name})

# Print the structured result
print("Parsed result:")
print(f"Title: {result['title']}")
print(f"Director: {result['director']}")
print(f"Year: {result['year']}")
print(f"Genre: {result['genre']}")

### Exercise 3
#### Working with Document Loaders and Text Splitters

You now know about about Document objects and how to load content from different sources. Now, let's implement a workflow to load documents, split them, and prepare them for retrieval.

**Instructions:**

1. Import the necessary document loaders to work with both PDF and web content.
2. Load the provided paper about LangChain architecture.
3. Create two different text splitters with varying parameters.
4. Compare the resulting chunks from different splitters.
5. Examine the metadata preservation across splitting.
6. Create a simple function to display statistics about your document chunks.



In [ ]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# Load the LangChain paper
paper_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf"
pdf_loader = PyPDFLoader(paper_url)
pdf_document = pdf_loader.load()

# Load content from LangChain website
web_url = "https://python.langchain.com/v0.2/docs/introduction/"
web_loader = WebBaseLoader(web_url)
web_document = web_loader.load()

# Create two different text splitters
splitter_1 = CharacterTextSplitter(chunk_size=300, chunk_overlap=30, separator="\n")
splitter_2 = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30, separators=["\n\n", "\n", " ", ""])

# Apply both splitters to the PDF document
chunks_1 = splitter_1.split_documents(pdf_document)
chunks_2 = splitter_2.split_documents(pdf_document)

# Define a function to display document statistics
def display_document_stats(docs, name):
    """Display statistics about a list of document chunks"""
    total_chunks = len(docs)
    total_chars = sum(len(doc.page_content) for doc in docs)
    avg_chunk_size = total_chars / total_chunks if total_chunks > 0 else 0

    # Count unique metadata keys across all documents
    all_metadata_keys = set()
    for doc in docs:
        all_metadata_keys.update(doc.metadata.keys())

    # Print the statistics
    print(f"\n=== {name} Statistics ===")
    print(f"Total number of chunks: {total_chunks}")
    print(f"Average chunk size: {avg_chunk_size:.2f} characters")
    print(f"Metadata keys preserved: {', '.join(all_metadata_keys)}")

    if docs:
        print("\nExample chunk:")
        example_doc = docs[min(5, total_chunks-1)]  # Get the 5th chunk or the last one if fewer
        print(f"Content (first 150 chars): {example_doc.page_content[:150]}...")
        print(f"Metadata: {example_doc.metadata}")

        # Calculate length distribution
        lengths = [len(doc.page_content) for doc in docs]
        min_len = min(lengths)
        max_len = max(lengths)
        print(f"Min chunk size: {min_len} characters")
        print(f"Max chunk size: {max_len} characters")

# Display stats for both chunk sets
display_document_stats(chunks_1, "Splitter 1")
display_document_stats(chunks_2, "Splitter 2")

#### Embedding models


Embedding models are specifically designed to interface with text embeddings.

Embeddings generate a vector representation for a specified piece or "chunk" of text.  Embeddings offer the advantage of allowing you to conceptualize text within a vector space. Consequently, you can perform operations such as semantic search, where you identify pieces of text that are most similar within the vector space.


IBM, OpenAI, Hugging Face, and others offer embedding models. Here, you will use the embedding model from IBM's watsonx.ai to work with the text.


In [ ]:
# Import the EmbedTextParamsMetaNames class from ibm_watsonx_ai.metanames module
# This class provides constants for configuring Watson embedding parameters
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames

# Configure embedding parameters using a dictionary:
# - TRUNCATE_INPUT_TOKENS: Limit the input to 3 tokens (very short, possibly for testing)
# - RETURN_OPTIONS: Request that the original input text be returned along with embeddings
embed_params = {
 EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
 EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

In [ ]:
# Import the WatsonxEmbeddings class from langchain_ibm module
# This provides an integration between LangChain and IBM's Watson AI services
from langchain_ibm import WatsonxEmbeddings

# Create a WatsonxEmbeddings instance with the following configuration:
# - model_id: Specifies the "slate-125m-english-rtrvr" embedding model from IBM
# - url: The endpoint URL for the Watson service in the US South region
# - project_id: The Watson project ID to use ("skills-network")
# - params: The embedding parameters configured earlier
watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    params=embed_params,
)

### Exercise 4
#### **Building a Simple Retrieval System with LangChain**

In this exercise, you'll implement a simple retrieval system using LangChain's vector store and retriever components to help answer questions based on a document.

**Instructions:**

1. Import the necessary components for document loading, embedding, and retrieval.
2. Load the provided document about artificial intelligence.
3. Split the document into manageable chunks.
4. Use an embedding model to create vector representations.
5. Create a vector store and a retriever.
6. Implement a simple question-answering system.
7. Test your system with at least 3 different questions.



In [ ]:
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain.chains import RetrievalQA

# 1. Load a document about AI
loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")
documents = loader.load()

# 2. Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# 3. Set up the embedding model. (Use an embedding model to create vector representations.)
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

embedding_model = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network"
    params=embed_params,
)

# 4. Create a vector store
vector_store = Chroma.from_documents(chunks, embedding_model)

# 5. Create a retriever
retriever = vector_store.as_retriever()

# 6. Define a function to search for relevant information
def search_documents(query, top_k=3):
    """Search for documents relevant to a query"""
    # Use the retriever to get relevant documents
    docs = retriever.get_relevant_documents(query)

    # Limit to top_k if specified
    return docs[:top_k]

# 7. Test with a few queries
test_queries = [
    "What is LangChain?",
    "How do retrievers work?",
    "Why is document splitting important?"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    results = search_documents(query)
    # Print the results
    print(f"Found {len(results)} documents:")
    for i, doc in enumerate(results):
        print(f"\nResult {i+1}: {doc.page_content[:100]}...")
        print(f"Source: {doc.metadata.get('source', 'Unknown')}")

### Exercise 5
#### **Building a Chatbot with Memory using LangChain**

In this exercise, you'll create a simple chatbot that can remember previous interactions using LangChain's memory components. You'll implement conversation memory to make your chatbot maintain context throughout a conversation.

**Instructions:**

1. Import the necessary components for chat history and conversation memory.
2. Set up a language model for your chatbot.
3. Create a conversation chain with memory capabilities.
4. Implement a simple interactive chat interface.
5. Test the memory capabilities with a series of related questions.
6. Examine how the conversation history is stored and accessed.



In [ ]:
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from langchain_core.messages import HumanMessage, AIMessage
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

# 1. Set up the language model
model_id = 'mistralai/mixtral-8x7b-instruct-v01'
parameters = {
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.2,
}
credentials = {"url": "https://us-south.ml.cloud.ibm.com"}
project_id = "skills-network"

# Initialize the model
model = ModelInference(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)
llm = WatsonxLLM(model=model)

# 2. Create a simple conversation with chat history
history = ChatMessageHistory()

# Add some initial messages (optional)
history.add_user_message("Hello, my name is Alice.")
history.add_ai_message("Hello, my name is AI Chatbot")

# 3. Print the current conversation history
for message in history.messages:
    sender = "AI" if isinstance(message, AIMessage) else "Human"
    print(f"{sender}: {message.content}")

# 4. Set up a conversation chain with memory
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# 5. Function to simulate a conversation
def chat_simulation(conversation, inputs):
    """Run a series of inputs through the conversation chain and display responses"""
    print("\n=== Beginning Chat Simulation ===")

    for i, user_input in enumerate(inputs):
        print(f"\n--- Turn {i+1} ---")
        print(f"Human: {user_input}")

        # Get response from the conversation chain
        response = conversation.invoke(input=user_input)

        # Print the AI's response
        print(f"AI: {response['response']}")

    print("\n=== End of Chat Simulation ===")

# 6. Test with a series of related questions
test_inputs = [
    "My favorite color is blue.",
    "I enjoy hiking in the mountains.",
    "What activities would you recommend for me?",
    "What was my favorite color again?",
    "Can you remember both my name and my favorite color?"
]

chat_simulation(conversation, test_inputs)

# 7. Examine the conversation memory
print("\nFinal Memory Contents:")
print(conversation.memory.buffer)

# 8. Create a new conversation with a different type of memory (optional)
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(llm=llm)
summary_conversation = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True
)

chat_simulation(summary_conversation, test_inputs)

print("\nFinal Memory Contents:")
print(summary_conversation.memory.buffer)

print("\n=== Compare Buffer Size ===")

# 9. Compare the buffer size of the two conversations
print(f"Buffer size of conversation: {len(conversation.memory.buffer)}")
print(f"Buffer size of summary conversation: {len(summary_conversation.memory.buffer)}")

### Exercise 6
#### **Implementing Multi-Step Processing with Different Chain Approaches**

In this exercise, you'll create a multi-step information processing system using both traditional chains and the modern LCEL approach. You'll build a system that analyzes product reviews, extracts key information, and generates responses based on the analysis.

**Instructions:**

1. Import the necessary components for both traditional chains and LCEL.
2. Implement a three-step process using both traditional SequentialChain and modern LCEL approaches.
3. Create templates for sentiment analysis, summarization, and response generation.
4. Test your implementations with sample product reviews.
5. Compare the flexibility and readability of both approaches.
6. Document the advantages and disadvantages of each method.



In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Sample product reviews for testing
positive_review = """I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing.
The built-in grinder saves me so much time in the morning, and the programmable timer means
I wake up to fresh coffee every day. Worth every penny and highly recommended to any coffee enthusiast."""

negative_review = """Disappointed with this laptop. It's constantly overheating after just 30 minutes of use,
and the battery life is nowhere near the 8 hours advertised - I barely get 3 hours.
The keyboard has already started sticking on several keys after just two weeks. Would not recommend to anyone."""

# Step 1: Define the prompt templates for each processing step
sentiment_template = """Analyze the sentiment of the following product review as positive, negative, or neutral.
Provide your analysis in the format: "SENTIMENT: [positive/negative/neutral]"

Review: {review}

Your analysis:
"""

summary_template = """Summarize the following product review into 3-5 key bullet points.
Each bullet point should be concise and capture an important aspect mentioned in the review.

Review: {review}
Sentiment: {sentiment}

Key points:
"""

response_template = """Write a helpful response to a customer based on their product review.
If the sentiment is positive, thank them for their feedback. If negative, express understanding
and suggest a solution or next steps. Personalize based on the specific points they mentioned.

Review: {review}
Sentiment: {sentiment}
Key points: {summary}

Response to customer:
"""

sentiment_prompt = PromptTemplate.from_template(sentiment_template)
summary_prompt = PromptTemplate.from_template(summary_template)
response_prompt = PromptTemplate.from_template(response_template)


# PART 1: Traditional Chain Approach
sentiment_chain = LLMChain(
    llm=mixtral_llm,
    prompt=sentiment_prompt,
    output_key="sentiment"
)
summary_chain = LLMChain(
    llm=mixtral_llm,
    prompt=summary_prompt,
    output_key="summary"
)
response_chain = LLMChain(
    llm=mixtral_llm,
    prompt=response_prompt,
    output_key="response"
)

overall_chain = SequentialChain(
    chains=[sentiment_chain, summary_chain, response_chain],
    input_variables=["review"],
    output_variables=["sentiment", "summary", "response"],
    verbose=True
)

# PART 2: LCEL Approach
sentiment_chain_lcel = sentiment_prompt | mixtral_llm | StrOutputParser()
summary_chain_lcel = summary_prompt | mixtral_llm | StrOutputParser()
response_chain_lcel = response_prompt | mixtral_llm | StrOutputParser()

overall_chain_lcel = (
    RunnablePassthrough.assign(
        sentiment=lambda x: sentiment_chain_lcel.invoke({"review": x["review"]})
    )
    | RunnablePassthrough.assign(
        summary=lambda x: summary_chain_lcel.invoke({
            "review": x["review"],
            "sentiment": x["sentiment"]
        })
    )
    | RunnablePassthrough.assign(
        response=lambda x: response_chain_lcel.invoke({
            "review": x["review"],
            "sentiment": x["sentiment"],
            "summary": x["summary"]
        })
    )
)

# Test both implementations
def test_chains(review):
    """Test both chain implementations with the given review"""
    print("\n" + "="*50)
    print(f"TESTING WITH REVIEW:\n{review[:100]}...\n")

    print("TRADITIONAL CHAIN RESULTS:")
    traditional_result = overall_chain.invoke({"review": review})
    print(f"Sentiment: {traditional_result['sentiment']}")
    print(f"Summary: {traditional_result['summary']}")
    print(f"Response: {traditional_result['response']}")

    print("\nLCEL CHAIN RESULTS:")
    lcel_result = overall_chain_lcel.invoke({"review": review})
    print(f"Sentiment: {lcel_result['sentiment']}")
    print(f"Summary: {lcel_result['summary']}")
    print(f"Response: {lcel_result['response']}")

    print("="*50)

# Run tests
test_chains(positive_review)
test_chains(negative_review)

### Exercise 7
#### **Creating Your First LangChain Agent with Basic Tools**

In this exercise, you'll build a simple agent that can help users with basic tasks using two custom tools. This exercise is a perfect starting point for understanding how LangChain agents work.

**Instructions:**

1. Create two simple tools: A calculator and a text formatter.
2. Set up a basic agent that can use these tools.
3. Test the agent with straightforward questions.



In [ ]:
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

def calculator(expression: str) -> str:
    """A simple calculator that can add, subtract, multiply, or divide two numbers.
    Input should be a mathematical expression like '2 + 2' or '15 / 3'."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

def format_text(text: str) -> str:
    """Format text to uppercase, lowercase, or title case.
    Input should be in format: '[format_type]: [text]'
    where format_type is 'uppercase', 'lowercase', or 'titlecase'."""
    try:
        # Handle empty input
        if not text or text.isspace():
            return "Error: Please provide some text to format."

        # Parse input to extract format type and content
        format_type, content = _parse_input(text)

        # Apply the specified formatting
        return _apply_formatting(format_type, content)
    except Exception as e:
        return f"Error formatting text: {str(e)}"

def _parse_input(text: str) -> tuple[str, str]:
    """Parse input to extract format type and content."""
    text = text.strip()

    # Check if input follows 'format: content' pattern
    if ":" in text and not text.startswith("http"):
        try:
            format_type, content = text.split(":", 1)
            return format_type.strip().lower(), content.strip()
        except ValueError:
            # Fallback if splitting fails
            return "titlecase", text
    else:
        # No format specified, default to titlecase
        return "titlecase", text


def _apply_formatting(format_type: str, content: str) -> str:
    """Apply the specified formatting to the content."""
    # Handle empty content
    if not content:
        return "Error: Please provide text content to format."

    # Define available formats and their corresponding functions
    format_functions = {
        "uppercase": str.upper,
        "lowercase": str.lower,
        "titlecase": str.title
    }

    # Apply formatting if format type is valid
    if format_type in format_functions:
        return format_functions[format_type](content)

    # Return helpful error message for invalid format types
    available_formats = ", ".join(format_functions.keys())
    return f"Unknown format type: '{format_type}'. Available formats: {available_formats}"

tools = [
    Tool(
        name="calculator",
        func=calculator,
        description="A simple calculator that can add, subtract, multiply, or divide two numbers."
    ),
    Tool(
        name="format_text",
        func=format_text,
        description="Format text to uppercase, lowercase, or title case. Input should be in format: '[format_type]: [text]' where format_type is 'uppercase', 'lowercase', or 'titlecase'."
    )
]

prompt_template = """You are a helpful assistant who can use tools to help with simple tasks.
You have access to these tools:

{tools}

The available tools you can use are: {tool_names}

Follow this format:

Question: the user's question
Thought: think about what to do, what tools to use, what to do after using the tool
Action: the tool to use, should be one of [{tool_names}]
Action Input: the input to the tool
Observation: the result from the tool
Thought: think about what you learned, what to do next
Final Answer: your final answer to the user's question

Question: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)
agent = create_react_agent(
    llm=mixtral_llm,
    tools=tools,
    prompt=prompt
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# Test with simple questions
test_questions = [
    "What is 25 + 63?",
    "Can you convert 'hello world' to uppercase?",
    "Calculate 15 * 7",
    "titlecase: langchain is awesome",
]


for question in test_questions:
    print(f"\n===== Testing: {question} =====")
    result = agent_executor.invoke({"input": question})
    print(f"Final Answer: {result['output']}")

© Copyright IBM Corporation. All rights reserved.
